# Convertible Bond Pricer Using QuantLib


In [22]:
!pip install QuantLib

In [4]:
import QuantLib as ql

In [5]:
calculation_date = ql.Date(22,2,2022)
ql.Settings.instance().evaluationDate = calculation_date

In [13]:
# St. Mary Land & Exploration Company 
# Bloomberg ticker: SM 5.75 03/15/22 

redemption = 100.00
face_amount = 100.0
spot_price = 3.02
conversion_price = 7.068
conversion_ratio = 14.14827

issue_date = ql.Date(30,1,2013)        
maturity_date = ql.Date(1,2,2033)

settlement_days = 2
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
coupon = 0.03
frequency = ql.Semiannual
tenor = ql.Period(frequency)

day_count = ql.Thirty360(ql.Thirty360.BondBasis)
accrual_convention = ql.Unadjusted
payment_convention = ql.Unadjusted

call_dates = [ql.Date(1,2,2019)]
call_price = 100.0
put_dates = [ql.Date(1,2,2019), ql.Date(1,2,2023), ql.Date(1,2,2028)]
put_price = 100.0

# assumptions
dividend_yield = 0
credit_spread_rate = 0.04
risk_free_rate = 0.04
volatility = 0.30

In [14]:
callability_schedule = ql.CallabilitySchedule()


for call_date in call_dates:
   callability_price  = ql.BondPrice(call_price, 
                                            ql.BondPrice.Clean)
   callability_schedule.append(ql.Callability(callability_price, 
                                       ql.Callability.Call,
                                       call_date)
                        )
    
for put_date in put_dates:
    puttability_price = ql.BondPrice(put_price, 
                                            ql.BondPrice.Clean)
    callability_schedule.append(ql.Callability(puttability_price,
                                               ql.Callability.Put,
                                               put_date))

In [15]:
dividend_schedule = ql.DividendSchedule() # No dividends
dividend_amount = dividend_yield*spot_price
next_dividend_date = ql.Date(1,12,2004)
dividend_amount = spot_price*dividend_yield
for i in range(4):
    date = calendar.advance(next_dividend_date, 1, ql.Years)
    dividend_schedule.append(
        ql.FixedDividend(dividend_amount, date)
    )

In [18]:
schedule = ql.Schedule(issue_date, maturity_date, tenor,
                       calendar, accrual_convention, accrual_convention,
                       ql.DateGeneration.Backward, False)

credit_spread_handle = ql.QuoteHandle(ql.SimpleQuote(credit_spread_rate))
exercise = ql.AmericanExercise(calculation_date, maturity_date)

convertible_bond = ql.ConvertibleFixedCouponBond(exercise,
                                                 conversion_ratio,
                                                 callability_schedule, 
                                                 issue_date,
                                                 settlement_days,
                                                 [coupon],
                                                 day_count,
                                                 schedule,
                                                 redemption)

In [19]:
spot_price_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
yield_ts_handle = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, day_count)
)
dividend_ts_handle = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_yield, day_count)
)
volatility_ts_handle = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar,volatility, day_count)
)

bsm_process = ql.BlackScholesMertonProcess(spot_price_handle, 
                                           dividend_ts_handle,
                                           yield_ts_handle,
                                           volatility_ts_handle)

In [20]:
time_steps = 1000
engine = ql.BinomialConvertibleEngine(bsm_process, "crr", time_steps, credit_spread_handle, dividend_schedule)

In [21]:
convertible_bond.setPricingEngine(engine)
print ("NPV ", convertible_bond.NPV())
print ("Accrued ", convertible_bond.accruedAmount())

NPV  96.59357075153648
Accrued  0.19166666666665666


# Greeks Calculation